#  MODIS Water Cluster Training

Version: 0.1.0

Date modified: 05.01.2023

Modified by: Amanda Burke

In [1]:
import csv
import datetime
import glob
import joblib
import numpy as np
import os
import math 
import pandas as pd
from pathlib import Path   
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split 

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import warnings


# plt.style.use('fivethirtyeight')
warnings.filterwarnings('ignore')
%matplotlib inline


import optuna
from sklearn.ensemble import RandomForestClassifier as skRF
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, f1_score
from sklearn.metrics import classification_report, roc_curve, auc, matthews_corrcoef
from sklearn.model_selection import RandomizedSearchCV, KFold, StratifiedKFold

# GPU-based frameworks
import cudf
import cupy as cp
from cuml.ensemble import RandomForestClassifier as cuRFC

In [2]:
GPU = True

In [3]:
MODEL = 'rf'
TEST_RATIO = 0.2
RANDOM_STATE = 42
LABEL_NAME = 'water'
if GPU is False:
    DATA_TYPE = np.int16
else: 
    DATA_TYPE = cp.float32
FRAC_LAND=0.5
num_datapoints = 10000000

In [4]:
#############################
# VERSION 4.2.1 (targeted 500k points)
TILE_IN = 'Golden'#v4.2.1
DATA_VERSION='v4.2.1'
offsets_indexes = ['x_offset', 'y_offset', 'year', 'julian_day','tileID']
#############################

# ##############################
# #VERSION 2.0.1 (5 million points)
# TILE_IN = 'GLOBAL'#v2.0.1
# DATA_VERSION='v2.0.1'
# offsets_indexes = ['x_offset', 'y_offset', 'year', 'julian_day']
# ##############################

# #############################
# #VERSION 0.0.0 (2billion data points)
# TILE_IN = 'cleaned'#v2.0.1
# DATA_VERSION='AGU'
# offsets_indexes = []#'x_offset', 'y_offset', 'year', 'julian_day']
# ##############################

training_data_basepath = f'/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/{DATA_VERSION}'
glob_string = os.path.join(training_data_basepath,'MOD*{}*.parquet.gzip'.format(TILE_IN))
data_paths = sorted([fv for fv in glob.glob(glob_string)])

#Only want the one with 4.2.0 because the other file doesnt work. 
print(data_paths)
data_path = data_paths[0]
print(data_path)

colsToDrop = [
    # 'sur_refl_b01_1',
    # 'sur_refl_b02_1',
    'sur_refl_b03_1',
    'sur_refl_b04_1','sur_refl_b05_1','sur_refl_b06_1',
    # 'sur_refl_b07_1',
    # 'ndvi',
    'ndwi1','ndwi2'
        ]

colsToDropTraining = colsToDrop.copy()
colsToDropTraining.extend(offsets_indexes)
v_names = ['sur_refl_b01_1','sur_refl_b02_1','sur_refl_b03_1',
           'sur_refl_b04_1','sur_refl_b05_1','sur_refl_b06_1',
           'sur_refl_b07_1','ndvi','ndwi1','ndwi2']

['/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/v4.2.1/MOD09_Golden_Masked_986161_4_2_1.parquet.gzip']
/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/v4.2.1/MOD09_Golden_Masked_986161_4_2_1.parquet.gzip


In [5]:
def load_cpu_data(fpath, colsToDrop, yCol='water', testSize=0.2, randomState=42, 
            dataType=np.float32, cpu=True, splitXY=False, trainTestSplit=False,
            applyLog=False, imbalance=False, frac=0.1, land=False, multi=False, 
            multisample=1000000):
    """
    Simple helper function for loading data to be used by models
    :param fpath: Path to the data to be ingested.
    :param dataType: Data type to convert ingested data to.
    :param colsToDrop: Columns which are not necessary, from which to drop.
    :param testSize: Ration to
    """
    if multi:
        all_dfs = [pd.read_csv(path_) for path_ in fpath]
        df = pd.concat(all_dfs).sample(n=multisample, random_state=randomState)
        print('DF length: {}'.format(len(df.index)))
    else:   
        df = pd.read_parquet(fpath) if '.parquet' in fpath else pd.read_csv(fpath)
    df = df[df['sur_refl_b01_1'] + df['sur_refl_b02_1'] != 0]
    df = df[df['sur_refl_b07_1'] + df['sur_refl_b02_1'] != 0]
    df = df[df['sur_refl_b06_1'] + df['sur_refl_b02_1'] != 0]

    df = df.drop(columns=colsToDrop)
    cleanedDF = df[~df.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0).astype(dataType)
    if applyLog:
        for col in cleanedDF.drop([yCol], axis=1).columns:
            print('Applying log1p func to {}'.format(col))
            cleanedDF[col] = np.log1p(cleanedDF[col])
        cleanedDF = cleanedDF[~cleanedDF.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0)
    df = None
    if imbalance:
        if land:
            print('Imbalancing data, sampling {} from water'.format(frac))
        else:
            print(f'Imbalancing data, sampling {frac} from land, {1-frac} from water')
        groupedDF = cleanedDF.groupby('water')
        dfs = [groupedDF.get_group(y) for y in groupedDF.groups]
        sampledDF = dfs[1].sample(frac=frac)if land else dfs[0].sample(frac=frac)
        concatDF = sampledDF.append(dfs[0]) if land else sampledDF.append(dfs[1])
        concatDF = concatDF.sample(frac=1)
        concatDF = concatDF.reset_index()
        cleanedDF = concatDF.drop(columns=['index'])
    if not splitXY:
        return cleanedDF
    X = cleanedDF.drop([yCol], axis=1).astype(dataType)
    y = cleanedDF[yCol].astype(dataType)
    if trainTestSplit:
        return train_test_split(X, y, test_size=TEST_RATIO)
    else:
        return X, y

In [6]:
def load_gpu_data(fpath, colsToDrop, yCol='water', testSize=0.2, randomState=42, 
            dataType=cp.float32, cpu=False, splitXY=True, trainTestSplit=True,
            applyLog=False, imbalance=False, frac=0.1, land=False, multi=False, 
            multisample=1000000):
    """
    Simple helper function for loading data to be used by models
    :param fpath: Path to the data to be ingested.
    :param dataType: Data type to convert ingested data to.
    :param colsToDrop: Columns which are not necessary, from which to drop.
    :param testSize: Ration to
    """
    if multi:
        all_dfs = [pd.read_csv(path_) for path_ in fpath]
        df = pd.concat(all_dfs).sample(n=multisample, random_state=randomState)
        print('DF length: {}'.format(len(df.index)))
    else:   
        df = pd.read_parquet(fpath) if '.parquet' in fpath else pd.read_csv(fpath)
    df = df[df['sur_refl_b01_1'] + df['sur_refl_b02_1'] != 0]
    df = df[df['sur_refl_b07_1'] + df['sur_refl_b02_1'] != 0]
    df = df[df['sur_refl_b06_1'] + df['sur_refl_b02_1'] != 0]
    df = df.drop(columns=colsToDrop)
    cleanedDF = df[~df.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0).astype(dataType)
    cleanedDF = cudf.from_pandas(cleanedDF) if not cpu else cleanedDF
    if applyLog:
        for col in cleanedDF.drop([yCol], axis=1).columns:
            print('Applying log1p func to {}'.format(col))
            cleanedDF[col] = np.log1p(cleanedDF[col])
        cleanedDF = cleanedDF[~cleanedDF.isin([np.NaN, np.inf, -np.inf]).any(1)].dropna(axis=0)
    df = None
    if imbalance:
        if land:
            print('Imbalancing data, sampling {} from water'.format(frac))
        else:
            print('Imbalancing data, sampling {} from land'.format(frac))
        groupedDF = cleanedDF.groupby('water')
        dfs = [groupedDF.get_group(y) for y in groupedDF.groups]
        sampledDF = dfs[1].sample(frac=frac)if land else dfs[0].sample(frac=frac)
        concatDF = sampledDF.append(dfs[0]) if land else sampledDF.append(dfs[1])
        concatDF = concatDF.sample(frac=1)
        concatDF = concatDF.reset_index()
        cleanedDF = concatDF.drop(columns=['index'])
    if not splitXY:
        return cleanedDF
    X = cleanedDF.drop([yCol], axis=1).astype(dataType)
    y = cleanedDF[yCol].astype(dataType)
    cleanedX = cleanedDF.drop([yCol], axis=1).astype(dataType)
    cleanedy = cleanedDF[yCol].astype(dataType)
    if trainTestSplit:
        return train_test_split(cleanedX, cleanedy, test_size=TEST_RATIO)
    else:
        return cleanedX, cleanedy

### Input data

In [8]:
colsToDrop

['sur_refl_b03_1',
 'sur_refl_b04_1',
 'sur_refl_b05_1',
 'sur_refl_b06_1',
 'ndwi1',
 'ndwi2']

In [9]:
%%time
load_data_params = {'fpath':data_path,'colsToDrop':colsToDropTraining,'splitXY':True,
                    'imbalance':False,'trainTestSplit':True}

# print(load_data_params)

if GPU is False: 
    X, X_test, y, y_test = load_cpu_data(**load_data_params)
else: 
    X, X_test, y, y_test = load_gpu_data(**load_data_params)
        # fpath=data_path,
        #  colsToDrop=colsToDropTraining,
        #  splitXY=True,
        #  imbalance=False,
        #  trainTestSplit=True)
        
X = X.iloc[:num_datapoints,:] 
y = y.iloc[:num_datapoints] 

X_test = X_test.iloc[:num_datapoints,:] 
y_test = y_test.iloc[:num_datapoints] 

print(f'data shape: {X.shape}, {y.shape}')

data shape: (788928, 4), (788928,)
CPU times: user 2.87 s, sys: 2.24 s, total: 5.11 s
Wall time: 32.3 s


In [10]:
X

,sur_refl_b01_1,sur_refl_b02_1,sur_refl_b07_1,ndvi
673826,34.0,-4.0,1.0,-12666.666992
296281,1551.0,1834.0,1411.0,836.041382
12470,583.0,2036.0,976.0,5547.000000
900642,34.0,0.0,54.0,-10000.000000
777584,416.0,144.0,30.0,-4857.143066
...,...,...,...,...
158742,501.0,1514.0,231.0,5027.295410
779159,2874.0,2889.0,1977.0,26.000000
879627,25.0,-4.0,20.0,-13809.523438
443314,322.0,3237.0,753.0,8190.502930


In [11]:
#Getting the indices that are associated with land (0) and water (1)
y_water_ind = np.where(y>0.5)[0]
y_land_ind = np.where(y<0.5)[0]

#Subset the X AND y data to later/ subset with the clusters and then combine for RFA
X_water = X.iloc[y_water_ind,:]
y_water = y.iloc[y_water_ind]

X_land = X.iloc[y_land_ind,:]
y_land = y.iloc[y_land_ind]

TypeError: no implementation found for 'numpy.where' on types that implement __array_function__: [<class 'cudf.core.series.Series'>]

In [ ]:
_ = [print(column) for column in X.columns]

In [ ]:
# DATA_VERSION='v4.2.1'
# training_data_basepath = f'/explore/nobackup/projects/ilab/data/MODIS/MODIS_WATER_ML/training_data/{DATA_VERSION}'

# #VERSION 4.2.1
# TILE_IN = 'Golden'#v4.2.1
# offsets_indexes = ['x_offset', 'y_offset', 'year', 'julian_day','tileID']

# glob_string = os.path.join(training_data_basepath,'MOD*{}*.parquet.gzip'.format(TILE_IN))
# data_paths = sorted([fv for fv in glob.glob(glob_string)])

# #Only want the one with 4.2.0 because the other file doesnt work. 
# print(data_paths)
# data_path = data_paths[0]
# print(data_path)
# colsToDropTraining = colsToDrop.copy()
# colsToDropTraining.extend(offsets_indexes)


# X_target, X_test_target, y_target, y_test_target = load_data(fpath=data_path,
#                                 colsToDrop=colsToDropTraining,
#                                 dataType=DATA_TYPE,
#                                 cpu=True,
#                                 splitXY=True,
#                                 trainTestSplit=True
#                                 )

# X_target = X_target.iloc[:num_datapoints,:] 
# y_target = y_target.iloc[:num_datapoints] 

# X_test_target = X_test_target.iloc[:num_datapoints,:] 
# y_test_target = y_test_target.iloc[:num_datapoints] 

# print(f'\n\ntarget subset data shape: {X_target.shape}, {y_target.shape}')

# #Getting the indices that are associated with land (0) and water (1)
# #Subset the X AND y data to later subset with the clusters and then combine for RFA
# X_water_target = X_target.iloc[np.where(y_target>0.5)[0],:]
# X_land_target = X_target.iloc[np.where(y_target<0.5)[0],:]

## Clustering Data for Input to Random Forest

Based on the cluster analysis above on 5.03.23, 15 clusters appears to have the most data and exclude outliers so will use that number for selection 

In [ ]:
CLUSTER_NUM=15

common_params = {
    "n_init": "auto",
    "random_state": 42,
    "init":"random"
}

In [ ]:
%%time
kme_land_random =  KMeans(n_clusters=CLUSTER_NUM, **common_params).fit(X_land)
kmeans_output_land_random = kme_land_random.predict(X_land)

In [ ]:
%%time
kme_water_random = KMeans(n_clusters=CLUSTER_NUM, **common_params).fit(X_water)
kmeans_output_water_random = kme_water_random.predict(X_water)

### Even Balanced Random pulled datapoints

In [ ]:
COUNT_EVEN_BALANCE_LAND = np.inf
COUNT_EVEN_BALANCE_WATER = np.inf
for cluster in np.unique(kmeans_output_water_random):
    land_num = len(np.where(kmeans_output_land_random == cluster)[0])
    water_num = len(np.where(kmeans_output_water_random == cluster)[0])
    if land_num < COUNT_EVEN_BALANCE_LAND: COUNT_EVEN_BALANCE_LAND = land_num
    if water_num < COUNT_EVEN_BALANCE_WATER: COUNT_EVEN_BALANCE_WATER = water_num
    
print(COUNT_EVEN_BALANCE_LAND, COUNT_EVEN_BALANCE_WATER)
if COUNT_EVEN_BALANCE_LAND < COUNT_EVEN_BALANCE_WATER:
    COUNT = COUNT_EVEN_BALANCE_LAND
else: 
    COUNT = COUNT_EVEN_BALANCE_WATER
print(COUNT,COUNT_EVEN_BALANCE_LAND,COUNT_EVEN_BALANCE_WATER)

In [ ]:
# np.random.seed(42)
random_ind_land = np.array([])
random_ind_water = []

for cluster in np.unique(kmeans_output_water_random):
    print(f'cluster {cluster}')
    cluster_ind_water = np.where(kmeans_output_water_random == cluster)[0]
    random_pts_water = np.random.choice(cluster_ind_water,COUNT,replace=False)
    max_X_random_water = np.nanmax(X_water['sur_refl_b01_1'].iloc[random_pts_water])
    if max_X_random_water < 10000:
        random_ind_water = np.append(random_ind_water, random_pts_water)
    else: print(f'Cluster {cluster} contains outliers')
    
    cluster_ind_land = np.where(kmeans_output_land_random == cluster)[0]
    random_pts_land = np.random.choice(cluster_ind_land,COUNT,replace=False)
    random_ind_land = np.append(random_ind_land, random_pts_land)
    
random_ind_water = random_ind_water.astype('int')
random_ind_land = random_ind_land.astype('int')

print(random_ind_water,random_ind_land)

### Percentage Random pulled datapoints

In [ ]:
# List of the clusters: kmeans_output_land and kmeans_output_water
# Data: X_water, X_land, y_water, y_land

PERCENT_RANDOM_PULL = 0.15

In [ ]:

np.random.seed(42)
random_ind_land = np.array([])
random_ind_water = []

for cluster in np.unique(kmeans_output_water_random):
    print(f'cluster {cluster}')
    cluster_ind_water = np.where(kmeans_output_water_random == cluster)[0]
    # cluster_ind_water = np.where(bgm_water == cluster)[0]
    COUNT_RANDOM_PULL_WATER = int(PERCENT_RANDOM_PULL*len(cluster_ind_water))
    random_pts_water = np.random.choice(cluster_ind_water,COUNT_RANDOM_PULL_WATER,replace=False)
    max_X_random_water = np.nanmax(X_water['sur_refl_b01_1'].iloc[random_pts_water])
    if max_X_random_water < 10000:
        random_ind_water = np.append(random_ind_water, random_pts_water)
    else: print(f'Cluster {cluster} contains outliers')
    
    cluster_ind_land = np.where(kmeans_output_land_random == cluster)[0]
    # cluster_ind_land = np.where(bgm_land == cluster)[0]
    COUNT_RANDOM_PULL_LAND = int(PERCENT_RANDOM_PULL*len(cluster_ind_land))
    random_pts_land = np.random.choice(cluster_ind_land,COUNT_RANDOM_PULL_LAND,replace=False)
    random_ind_land = np.append(random_ind_land, random_pts_land)
    # print(f'Pulling {COUNT_RANDOM_PULL_WATER} Water pts and {COUNT_RANDOM_PULL_LAND} Land pts')
    # print()
random_ind_water = random_ind_water.astype('int')
random_ind_land = random_ind_land.astype('int')

print(len(random_ind_water),len(random_ind_land))


In [ ]:
# fig, ax = plt.subplots(2, 2,figsize=(20, 10))
# var=0
# for col in range(2):
#     ax[col, 0].set_ylabel('Frequency') 
#     for row in range(2):
#         variable=X_cpu.columns[var]
#         if 'ndvi' in variable: 
#             continue
#             var_bins = bin_boundaries
#             log_values = False
#         else: 
#             var_bins = None
#             log_values = True
#         ax[row, col].hist(
#             [   
#             X_cpu[variable][not_same_point.index].values
#             ],
#             label=[
#             "data"
#             ],
#             bins=var_bins,
#         color=['brown'], log=log_values) 
#         ax[row, col].set_xlabel(f'{variable}')
#         var+=1
#     ax[0,0].legend(loc='upper right',fontsize=20)
# plt.show()

### Total random dataset used for training random forest

In [ ]:
X_cluster_land_random = X_land.iloc[random_ind_land]
y_cluster_land_random = y_land.iloc[random_ind_land]
X_cluster_water_random = X_water.iloc[random_ind_water]
y_cluster_water_random = y_water.iloc[random_ind_water]

X_cluster_random = pd.concat([X_cluster_land_random,X_cluster_water_random])
y_cluster_random = pd.concat([y_cluster_land_random,y_cluster_water_random])

#Combine the data so that we can shuffle the indices and keep the data together that should be
All_data_random = pd.concat([X_cluster_random,y_cluster_random],axis=1).sample(frac=1)

X_cluster_rfa_random = All_data_random[X_cluster_random.columns]
y_cluster_rfa_random = All_data_random['water']

print(X_cluster_rfa_random)
print(y_cluster_rfa_random)

In [ ]:
match_ind_land = np.random.choice(
    np.arange(len(X_land)),len(random_ind_land),replace=False)
print(random_ind_land)
print(match_ind_land)

match_ind_water = np.random.choice(
    np.arange(len(X_water)),len(random_ind_water),replace=False)
print(len(random_ind_water))
print(len(match_ind_water))

# X_match_land_random = X_land.iloc[match_ind_land]
# y_match_land_random = y_land.iloc[match_ind_land]
# X_match_water_random = X_water.iloc[match_ind_water]
# y_match_water_random = y_water.iloc[match_ind_water]

X_match_land_random = X_land.iloc[random_ind_land]
y_match_land_random = y_land.iloc[random_ind_land]
X_match_water_random = X_water.iloc[random_ind_water]
y_match_water_random = y_water.iloc[random_ind_water]

X_match_random = pd.concat([X_match_land_random,X_match_water_random])
y_match_random = pd.concat([y_match_land_random,y_match_water_random])

#Combine the data so that we can shuffle the indices and keep the data together that should be
All_data_match_random = pd.concat([X_match_random,y_match_random],axis=1).sample(frac=1).reset_index(drop=True)

X_match_rfa_random = All_data_match_random[X_match_random.columns]
y_match_rfa_random = All_data_match_random['water']

In [ ]:
# print(All_data_random)
print(X_match_rfa_random)
print(y_match_rfa_random)

### Plotting paramater space

In [ ]:
%%time

kme_land_random =  KMeans(n_clusters=CLUSTER_NUM, **common_params).fit(X_land)
kmeans_output_land_random = kme_land_random.predict(X_land)
kme_water_random = KMeans(n_clusters=CLUSTER_NUM, **common_params).fit(X_water)
kmeans_output_water_random = kme_water_random.predict(X_water)

In [ ]:
%%time

np.random.seed(42)
random_ind_land_eb = np.array([])
random_ind_water_eb = []

for cluster in np.unique(kmeans_output_water_random):
    print(f'cluster {cluster}')
    cluster_ind_water = np.where(kmeans_output_water_random == cluster)[0]
    random_pts_water = np.random.choice(cluster_ind_water,COUNT,replace=False)
    max_X_random_water = np.nanmax(X_water['sur_refl_b01_1'].iloc[random_pts_water])
    if max_X_random_water < 10000:
        random_ind_water_eb = np.append(random_ind_water_eb, random_pts_water)
    else: print(f'Cluster {cluster} contains outliers')
    
    cluster_ind_land = np.where(kmeans_output_land_random == cluster)[0]
    random_pts_land = np.random.choice(cluster_ind_land,COUNT,replace=False)
    random_ind_land_eb = np.append(random_ind_land_eb, random_pts_land)
    
random_ind_water_eb = random_ind_water_eb.astype('int')
random_ind_land_eb = random_ind_land_eb.astype('int')

print(random_ind_water_eb,random_ind_land_eb)


#############

match_ind_land_eb = np.random.choice(np.arange(len(X_land)),len(random_ind_land_eb),replace=False)
match_ind_water_eb = np.random.choice(np.arange(len(X_water)),len(random_ind_water_eb),replace=False)

X_match_land_eb = X_land.iloc[match_ind_land_eb]
X_match_water_eb = X_water.iloc[match_ind_water_eb]
X_match_eb = pd.concat([X_match_land_eb,X_match_water_eb])

X_cluster_land_eb = X_land.iloc[random_ind_land_eb]
X_cluster_water_eb = X_water.iloc[random_ind_water_eb]
X_cluster_eb = pd.concat([X_cluster_land_eb,X_cluster_water_eb])




In [ ]:
%%time


#############
np.random.seed(42)
random_ind_land_p = np.array([])
random_ind_water_p= []

for cluster in np.unique(kmeans_output_water_random):
    print(f'cluster {cluster}')
    cluster_ind_water = np.where(kmeans_output_water_random == cluster)[0]
    # cluster_ind_water = np.where(bgm_water == cluster)[0]
    COUNT_RANDOM_PULL_WATER = int(PERCENT_RANDOM_PULL*len(cluster_ind_water))
    random_pts_water = np.random.choice(cluster_ind_water,COUNT_RANDOM_PULL_WATER,replace=False)
    max_X_random_water = np.nanmax(X_water['sur_refl_b01_1'].iloc[random_pts_water])
    if max_X_random_water < 10000:
        random_ind_water_p = np.append(random_ind_water_p, random_pts_water)
    else: print(f'Cluster {cluster} contains outliers')
    
    cluster_ind_land = np.where(kmeans_output_land_random == cluster)[0]
    # cluster_ind_land = np.where(bgm_land == cluster)[0]
    COUNT_RANDOM_PULL_LAND = int(PERCENT_RANDOM_PULL*len(cluster_ind_land))
    random_pts_land = np.random.choice(cluster_ind_land,COUNT_RANDOM_PULL_LAND,replace=False)
    random_ind_land_p = np.append(random_ind_land_p, random_pts_land)
    print(f'Pulling {COUNT_RANDOM_PULL_WATER} Water pts and {COUNT_RANDOM_PULL_LAND} Land pts')
    print()
random_ind_water_p = random_ind_water_p.astype('int')
random_ind_land_p = random_ind_land_p.astype('int')

print(random_ind_water_p,random_ind_land_p)

#############

match_ind_land_p = np.random.choice(np.arange(len(X_land)),len(random_ind_land_p),replace=False)
match_ind_water_p = np.random.choice(np.arange(len(X_water)),len(random_ind_water_p),replace=False)

X_match_land_p = X_land.iloc[match_ind_land_p]
X_match_water_p = X_water.iloc[match_ind_water_p]
X_match_p = pd.concat([X_match_land_p,X_match_water_p])

X_cluster_land_p = X_land.iloc[random_ind_land_p]
X_cluster_water_p = X_water.iloc[random_ind_water_p]
X_cluster_p = pd.concat([X_cluster_land_p,X_cluster_water_p])

In [ ]:
fig, ax = plt.subplots(2, 2,figsize=(20, 10))
var=0
for col in range(2):
    ax[col, 0].set_ylabel('Frequency') 
    for row in range(2):
        variable=X_land.columns[var]
        if 'ndvi' in variable: 
            # var_bins = bin_boundaries
            log_values = False
        else: 
            # var_bins = None
            log_values = True
        ax[row, col].hist(
            [  
            # X_cluster_eb[variable].values,
            # X_match_eb[variable].values,
            X_cluster_p[variable].values,
            X_match_p[variable].values,
            ],
            label=[
            # f"EB Cluster {len(X_cluster_eb)}",
            # "EB Match",
            f"P Cluster {len(X_match_p)}",
            f"P Match"
            ],
            #bins=var_bins,
        #color=['darkgreen','lightgreen','darkblue','lightblue'], log=log_values) 
        color=['plum','darkorchid'], log=log_values) 
        ax[row, col].set_xlabel(f'{variable}')
        var+=1
    ax[0,0].legend(loc='upper right',fontsize=20)
plt.show()

In [ ]:
fig, ax =  plt.subplots(1, 1,figsize=(10, 5))
variable = X_land.columns[0]

plt.hist(
    [X_cluster_p[variable].values,
     X_match_p[variable].values,
    ],
    label=[
        f"P Cluster {len(X_match_p)}",
        f"P Match"
        ],
    color=['plum','darkorchid'], log=True) 

plt.ylabel('Frequency') 
plt.xlabel(f'{variable}')
plt.legend(loc='upper right',fontsize=20)   
plt.show()


# Random forest

In [12]:
def cpu_rf_objective(trial):
    list_trees = [75, 100, 125, 150, 175, 200, 250, 300, 400, 500]
    max_depth = [5, 10, 30, 50, 80, 90, 100, 110]
    min_samples_leaf = [1, 2, 3, 4, 5]
    min_samples_split = [2, 4, 8, 10]
    bootstrap = [True, False]
    max_features = ['auto', 'sqrt', 'log2']
    
    param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
       'max_depth':trial.suggest_categorical('max_depth', max_depth), 
       'min_samples_split':trial.suggest_categorical('min_samples_split', min_samples_split), 
       'min_samples_leaf':trial.suggest_categorical('min_samples_leaf', min_samples_leaf), 
       'bootstrap': trial.suggest_categorical('bootstrap', bootstrap),
       'criterion':'gini', 
       #'min_weight_fraction_leaf': trial.suggest_float('min_weight_fraction_leaf', 1e-8, 1.0, log=True), 
       'max_features':trial.suggest_categorical('max_features', max_features), 
       'max_leaf_nodes':None, 
       'min_impurity_decrease':0.0, 
       'oob_score':False, 
       'n_jobs':-1, 
       # 'random_state':42, 
       'verbose':0, 
       'warm_start':False, 
       'class_weight':None, 
       'ccp_alpha':0.0, 
       'max_samples':None
        }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    for idx, (train_idx, val_idx) in enumerate(cv.split(X,y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = skRF(**param)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        cv_scores[idx] = f1_score(y_val, preds)
        if cv_scores[idx] == 0.0:
            print('Pruning because of 0.0 score.')
            return 0.0
        print('Fold {}: {}'.format(idx, cv_scores[idx]))
    return np.mean(cv_scores)

search_space={
    "n_estimators": [75, 100, 125, 150, 175, 200, 250, 300, 400, 500],
    "max_depth" : [5,10, 30, 50, 80, 90, 100, 110],
    "min_samples_leaf" : [1, 2, 3, 4, 5],
    "min_samples_split" : [2, 4, 8, 10],
    "bootstrap" : [True, False],
    "max_features" : ['auto', 'sqrt', 'log2'],
    
}
TREES_AND_DEPTH_ONLY = False
GRID_SEARCH = True

In [13]:
def gpu_rf_objective(trial):
    list_trees = [75, 100, 125, 150, 175, 200, 250, 300, 400, 500]
    max_depth = [5, 10, 30, 50, 80, 90, 100, 110]
    min_samples_leaf = [1, 2, 3, 4, 5]
    min_samples_split = [2, 4, 8, 10]
    bootstrap = [True, False]
    max_features = ['auto', 'sqrt', 'log2']
    
    param = {'n_estimators': trial.suggest_categorical('n_estimators', list_trees), 
        'max_depth':trial.suggest_categorical('max_depth', max_depth), 
        'min_samples_split':trial.suggest_categorical('min_samples_split', min_samples_split), 
        'min_samples_leaf':trial.suggest_categorical('min_samples_leaf', min_samples_leaf), 
        'max_features':trial.suggest_categorical('max_features', max_features), 
            }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    #######################
    # HERE IS WHERE TO CHANGE THE X,Y DATASET USED FOR TRAINING
    #######################
   
    cv_scores = np.empty(5)
    for idx, (train_idx, val_idx) in enumerate(cv.split(X.to_pandas(),y.to_pandas())):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        model = cuRFC(**param)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        cv_scores[idx] = f1_score(y_val.to_numpy(), preds.to_numpy())
        del model, preds
        if cv_scores[idx] == 0.0:
            print('Pruning because of 0.0 score.')
            return 0.0
        print('Fold {}: {}'.format(idx, cv_scores[idx]))
    return np.mean(cv_scores)
    
search_space={
    "n_estimators": [75, 100, 125, 150, 175, 200, 250, 300, 400, 500],
    "max_depth" : [5,10, 30, 50, 80, 90, 100, 110],
    "min_samples_leaf" : [1, 2, 3, 4, 5],
    "min_samples_split" : [2, 4, 8, 10],
    "bootstrap" : [True, False],
    "max_features" : ['auto', 'sqrt', 'log2'],
    
}
TREES_AND_DEPTH_ONLY = False
GRID_SEARCH = True

In [14]:
%%time

optuna.logging.set_verbosity(optuna.logging.INFO)
if GRID_SEARCH:
    study = optuna.create_study(study_name='RF Tuning Grid Search', 
                                direction='maximize',
                                sampler=optuna.samplers.GridSampler(search_space))
    
else:
    study = optuna.create_study(study_name='RF Tuning',
                                direction='maximize')
#Objective is under the functions area

#####################################################################
#CHANGE HERE FOR DIFFERENT MODELING TYPE
#rf_objective or xgb_objective
#####################################################################
if GPU is False:
    study.optimize(cpu_rf_objective, n_trials=25, timeout=30*600)
else: 
    study.optimize(gpu_rf_objective, n_trials=25, timeout=30*600)

[I 2023-11-09 15:44:18,593] A new study created in memory with name: RF Tuning Grid Search


Fold 0: 0.9666724776970128
Fold 1: 0.9673792653507108


[W 2023-11-09 15:44:38,598] Trial 0 failed with parameters: {'n_estimators': 250, 'max_depth': 80, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_3273859/3230189718.py", line 27, in gpu_rf_objective
    model.fit(X_train, y_train)
  File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/contextlib.py", line 75, in inner
    return func(*args, **kwds)
  File "/panfs/ccds02/app/modules/jupyter/ilab/tensorflow-kernel/lib/python3.8/site-packages/cuml/internals/api_decorators.py", line 190, in wrapper
    ret = func(*args, **kwargs)
KeyboardInterrupt
[W 2023-11-09 15:44:38,603] Trial 0 failed with value None.


KeyboardInterrupt: 

#### Training and output best model

In [ ]:
trials = study.best_trials            
max_trial_score = max([trial.values[0] for trial in trials])
max_trial_params = [trial.params for trial in trials 
                        if trial.values[0] == max_trial_score][0]
max_trial_params['n_jobs'] = -1
score_print = int(np.round(max_trial_score,4)*1000)
print(max_trial_score)
print(score_print)

In [ ]:
hyperparameters = max_trial_params
hyperparameters['n_jobs'] = -1
print('Using these params:')
print(hyperparameters)
tuned_classifier = skRF(**hyperparameters)

In [ ]:
%%time 
tuned_classifier.fit(X,y)

In [ ]:
# import pickle
# # save the model to disk
# filename = f'rfa_models/MODIS_RFA_Targeted_v000_MaxScore{score_print}_sfcref127ndvi.pkl'
# print(filename)
# pickle.dump(tuned_classifier, open(filename, 'wb'))

In [ ]:
# pickled_model = pickle.load(open('rfa_models/MODIS_RFA_v201_EBCluster_sfcref127ndvi_4.pkl', 'rb'))
# print(pickled_model)